Importar las librerías necesarias

In [190]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC  # Importa el modelo SVM
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
import spacy

# Cargar modelo de idioma de spaCy
nlp = spacy.load("en_core_web_sm")

Cargar los datos

In [191]:
df = pd.read_csv("../data/youtube_cln_proc.csv", index_col = False)
df.head(2)

,IsToxic,processed_text
0,False,people step case not people situation lump mes...
1,True,law enforcement train shoot apprehend train sh...


Vectorización del texto

In [192]:
# Vectorización con TF-IDF
vectorizer = TfidfVectorizer(max_features = 500)
X = vectorizer.fit_transform(df['processed_text'])

# Definir la variable objetivo (target)
y = df['IsToxic']  

print(f"X: {X}\n" )
print(f"y: {y}\n" )

X: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8557 stored elements and shape (999, 500)>
  Coords	Values
  (0, 312)	0.2960914997024088
  (0, 413)	0.12713387022453032
  (0, 52)	0.09758696518986769
  (0, 292)	0.056703007377903444
  (0, 396)	0.10714668486618889
  (0, 262)	0.0931458310436694
  (0, 174)	0.09758696518986769
  (0, 257)	0.0983022144484625
  (0, 220)	0.12713387022453032
  (0, 335)	0.1819253914968009
  (0, 172)	0.08077545389923979
  (0, 461)	0.07951056926517955
  (0, 470)	0.08672754186756736
  (0, 180)	0.21429336973237778
  (0, 155)	0.07920548529882318
  (0, 216)	0.12456465143420471
  (0, 436)	0.08143650897340117
  (0, 231)	0.18089884305109183
  (0, 261)	0.11492688821125402
  (0, 427)	0.1201873918519915
  (0, 62)	0.10944492938907753
  (0, 119)	0.12456465143420471
  (0, 200)	0.24453281382263214
  (0, 178)	0.21216483558705526
  (0, 473)	0.18745509892909967
  :	:
  (992, 176)	0.3286228169810456
  (992, 432)	0.33208766830004055
  (993, 431)	0.48178088301956895
  (

Dividir el dataset en entrenamiento y prueba

In [193]:
#  Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Crear y Entrenar el modelo SVM

In [194]:
# Entrenar el modelo SVM
svm_model = SVC(kernel='rbf', C=0.3)  # Puedes cambiar el kernel a 'rbf', 'poly', etc., si deseas experimentar
svm_model.fit(X_train, y_train)

SVC(C=0.3)

### Evaluación y Detección de Overfitting

Comprobar el rendimiento del modelo en los conjuntos de entrenamiento y prueba.

In [195]:
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)

# Calcular métricas
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
overfitting = train_accuracy - test_accuracy

# Imprimir resultados
print(f"Accuracy en entrenamiento: {train_accuracy * 100:.2f}%")
print(f"Accuracy en prueba: {test_accuracy * 100:.2f}%")
print(f"Overfitting: {overfitting * 100:.2f}%")
print("\nReporte de clasificación detallado:")
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix:")
print(conf_matrix)


Accuracy en entrenamiento: 62.66%
Accuracy en prueba: 53.33%
Overfitting: 9.33%

Reporte de clasificación detallado:
              precision    recall  f1-score   support

       False       0.52      1.00      0.69       153
        True       1.00      0.05      0.09       147

    accuracy                           0.53       300
   macro avg       0.76      0.52      0.39       300
weighted avg       0.76      0.53      0.39       300

Confusion Matrix:
[[153   0]
 [140   7]]


### Evaluación Adicional y Reducción del Overfitting

Usar validación cruzada y, en caso de overfitting alto, podemos aplicar técnicas de ajuste como selección de características o regularización.

In [196]:
# Validación cruzada para evaluar el modelo y reducir overfitting
cv_scores = cross_val_score(svm_model, X, y, cv=5)
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Promedio de Cross-Validation Accuracy: {cv_scores.mean() * 100:.2f}%")

Cross-Validation Accuracy Scores: [0.59       0.565      0.59       0.615      0.57788945]
Promedio de Cross-Validation Accuracy: 58.76%


In [197]:
# Entrenar el modelo en el conjunto de entrenamiento completo y evaluar en el conjunto de prueba
svm_model.fit(X_train, y_train)
y_train_pred = svm_model.predict(X_train)  # Predicciones en entrenamiento
y_test_pred = svm_model.predict(X_test)    # Predicciones en prueba

# Evaluar el modelo en entrenamiento y prueba
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
class_report = classification_report(y_test, y_test_pred)
overfitting = train_accuracy - test_accuracy

# Imprimir resultados
print(f"Accuracy en entrenamiento: {train_accuracy * 100:.2f}%")
print(f"Accuracy en prueba: {test_accuracy * 100:.2f}%")
print(f"Overfitting: {overfitting * 100:.2f}%")
print("\nReporte de clasificación detallado:")
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix:")
print(conf_matrix)

# Comparación de resultados para detectar overfitting
if train_accuracy - test_accuracy > 0.05:
    print("\nPosible overfitting detectado: el modelo rinde mucho mejor en entrenamiento que en prueba.")
else:
    print("\nNo se detectó un sobreajuste significativo.")

Accuracy en entrenamiento: 62.66%
Accuracy en prueba: 53.33%
Overfitting: 9.33%

Reporte de clasificación detallado:
              precision    recall  f1-score   support

       False       0.52      1.00      0.69       153
        True       1.00      0.05      0.09       147

    accuracy                           0.53       300
   macro avg       0.76      0.52      0.39       300
weighted avg       0.76      0.53      0.39       300

Confusion Matrix:
[[153   0]
 [140   7]]

Posible overfitting detectado: el modelo rinde mucho mejor en entrenamiento que en prueba.


# Ajustar los Hiperparámetros del Modelo

In [198]:
# Primero, hacer el split con los datos sin vectorizar
X_train, X_test, y_train, y_test = train_test_split(
    df['processed_text'],  # ← Usa el texto procesado sin vectorizar
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [199]:
# Crear pipeline incluyendo TANTO el vectorizador como el clasificador
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # ← Añade el vectorizador al pipeline
    ('clf', SVC(random_state=42, max_iter=2000))
])

In [200]:
# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'tfidf__max_features': [500, 1000, 1500],  # ← Añade el prefijo 'tfidf__'
    'tfidf__ngram_range': [(1,1), (1,2)],      # ← Añade el prefijo 'tfidf__'
    'tfidf__min_df': [2, 3, 5],                # ← Añade el prefijo 'tfidf__'
    'clf__C': [0.1, 1, 10, 100],               # Valores de C para regularización
    'clf__kernel': ['linear', 'rbf', 'poly'],   # Tipos de kernel a probar
    'clf__gamma': ['scale', 'auto']             # Opciones de gamma para kernels no lineales
}

In [201]:
# Crear y ejecutar GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

In [202]:
# Ajustar el modelo
print("Iniciando búsqueda de mejores hiperparámetros...")
grid_search.fit(X_train, y_train)

Iniciando búsqueda de mejores hiperparámetros...
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


c:\Proyectos_F5\Grupo6_LNP\Grupo6_NLP\venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf',
                                        SVC(max_iter=2000, random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.1, 1, 10, 100],
                         'clf__gamma': ['scale', 'auto'],
                         'clf__kernel': ['linear', 'rbf', 'poly'],
                         'tfidf__max_features': [500, 1000, 1500],
                         'tfidf__min_df': [2, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy', verbose=1)

In [203]:
# Resultados
print("Mejores hiperparámetros encontrados:", grid_search.best_params_)
print("Mejor puntuación de validación:", grid_search.best_score_)

Mejores hiperparámetros encontrados: {'clf__C': 1, 'clf__gamma': 'scale', 'clf__kernel': 'linear', 'tfidf__max_features': 1500, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Mejor puntuación de validación: 0.7146383647798742


Crear y entrenar el modelo con los mejores hiperparámetros

In [204]:
# Crear modelo final con los mejores parámetros
best_tfidf = TfidfVectorizer(
    max_features=grid_search.best_params_['tfidf__max_features'],
    ngram_range=grid_search.best_params_['tfidf__ngram_range'],
    min_df=grid_search.best_params_['tfidf__min_df']
)

best_model = SVC(
    C=grid_search.best_params_['clf__C'],
    kernel=grid_search.best_params_['clf__kernel'],
    gamma=grid_search.best_params_['clf__gamma'],
    random_state=42,
    max_iter=2000
)

In [205]:
# Entrenar modelo final
X_train_vectorized = best_tfidf.fit_transform(X_train)
X_test_vectorized = best_tfidf.transform(X_test)
best_model.fit(X_train_vectorized, y_train)

SVC(C=1, kernel='linear', max_iter=2000, random_state=42)

Evaluar el modelo final

In [206]:
#  Evaluar resultados
y_train_pred = best_model.predict(X_train_vectorized)
y_test_pred = best_model.predict(X_test_vectorized)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
overfitting = train_accuracy - test_accuracy

print("\nResultados finales:")
print(f"Accuracy en entrenamiento: {train_accuracy * 100:.2f}%")
print(f"Accuracy en prueba: {test_accuracy * 100:.2f}%")
print(f"Overfitting: {overfitting * 100:.2f}%")
print("\nReporte de clasificación detallado:")
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix:")
print(conf_matrix)


Resultados finales:
Accuracy en entrenamiento: 93.87%
Accuracy en prueba: 68.00%
Overfitting: 25.87%

Reporte de clasificación detallado:
              precision    recall  f1-score   support

       False       0.70      0.72      0.71       108
        True       0.66      0.63      0.64        92

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.68       200
weighted avg       0.68      0.68      0.68       200

Confusion Matrix:
[[78 30]
 [34 58]]


Usar Regularización L2 para reducir el overfitting LinealSVC

In [207]:
# Importar las librerías necesarias
from sklearn.svm import LinearSVC

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df['processed_text'],  # Texto procesado sin vectorizar
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Crear pipeline que incluya el vectorizador y el clasificador LinearSVC
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC(random_state=42, max_iter=2000))
])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'tfidf__max_features': [500, 1000, 1500],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__min_df': [2, 3, 5],
    'clf__C': [0.01, 0.1, 1, 10, 100]  # Valores de C para ajustar regularización L2
}

# Crear y ejecutar GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Ajustar el modelo
print("Iniciando búsqueda de mejores hiperparámetros...")
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:", grid_search.best_params_)

# Evaluar el modelo en el conjunto de prueba
y_pred = grid_search.predict(X_test)
print("Reporte de clasificación en conjunto de prueba:")
print(classification_report(y_test, y_pred))


Iniciando búsqueda de mejores hiperparámetros...
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Mejores hiperparámetros encontrados: {'clf__C': 0.1, 'tfidf__max_features': 1000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1)}
Reporte de clasificación en conjunto de prueba:
              precision    recall  f1-score   support

       False       0.66      0.79      0.72       108
        True       0.68      0.52      0.59        92

    accuracy                           0.67       200
   macro avg       0.67      0.65      0.65       200
weighted avg       0.67      0.67      0.66       200



In [209]:
# Crear modelo final con los mejores parámetros
best_tfidf = TfidfVectorizer(
    max_features=grid_search.best_params_['tfidf__max_features'],
    ngram_range=grid_search.best_params_['tfidf__ngram_range'],
    min_df=grid_search.best_params_['tfidf__min_df']
)

best_model = LinearSVC(
    C=grid_search.best_params_['clf__C'],
    random_state=42,
    max_iter=2000
)

In [210]:
# Entrenar modelo final
X_train_vectorized = best_tfidf.fit_transform(X_train)
X_test_vectorized = best_tfidf.transform(X_test)
best_model.fit(X_train_vectorized, y_train)

LinearSVC(C=0.1, max_iter=2000, random_state=42)

In [211]:
#  Evaluar resultados
y_train_pred = best_model.predict(X_train_vectorized)
y_test_pred = best_model.predict(X_test_vectorized)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
overfitting = train_accuracy - test_accuracy

print("\nResultados finales:")
print(f"Accuracy en entrenamiento: {train_accuracy * 100:.2f}%")
print(f"Accuracy en prueba: {test_accuracy * 100:.2f}%")
print(f"Overfitting: {overfitting * 100:.2f}%")
print("\nReporte de clasificación detallado:")
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix:")
print(conf_matrix)


Resultados finales:
Accuracy en entrenamiento: 88.86%
Accuracy en prueba: 66.50%
Overfitting: 22.36%

Reporte de clasificación detallado:
              precision    recall  f1-score   support

       False       0.66      0.79      0.72       108
        True       0.68      0.52      0.59        92

    accuracy                           0.67       200
   macro avg       0.67      0.65      0.65       200
weighted avg       0.67      0.67      0.66       200

Confusion Matrix:
[[85 23]
 [44 48]]
